### Librerias

In [1]:
import cv2
import os
import numpy as np
import mediapipe as mp

### Funcion Guardar Fotos de un Video

In [2]:
import os
import cv2

def save_frames_from_video(video_path, output_folder, frame_size=(100, 100), frame_interval=10):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Resize the frame
        resized_frame = cv2.resize(gray_frame, frame_size)

        frame_filename = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)

        # Save the grayscale frame
        cv2.imwrite(frame_path, resized_frame)

        frame_count += 1

        # Skip 'frame_interval' number of frames
        for _ in range(frame_interval):
            ret, _ = cap.read()
            if not ret:
                break

    print(f"{frame_count} fotos guardadas en {output_folder}")
    cap.release()

# Example usage:
# save_frames_from_video('path/to/video.mp4', 'path/to/output/folder')


### Funcion Para Aplicar el Haarcascade V1

In [3]:
def buscando_a_nemo(video_path, cascade_path, scale_factor=1.1, min_neighbors=5, min_size=32):
    clasificador = cv2.CascadeClassifier(cascade_path)

    cap = cv2.VideoCapture(video_path)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect objects in the frame
        objects = clasificador.detectMultiScale(gray, scaleFactor=scale_factor, minNeighbors=min_neighbors, minSize=(min_size, min_size))

        # Draw rectangles around the detected objects
        for (x, y, w, h) in objects:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.imshow('deteccion', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

### Funcion Para Aplicar el Haarcascade V2

In [4]:
def buscando_a_dori(video_path, cascade_path):
    mp_face_detection = mp.solutions.face_detection
    
    LABELS = ["Con_mascarilla", "Sin_mascarilla"]
    
    face_mask = cv2.face.LBPHFaceRecognizer_create()
    face_mask.read(cascade_path)
    
    # cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    cap = cv2.VideoCapture(video_path)
    
    with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
        
        while True:
            ret, frame = cap.read()
            if ret == False: break
            
            height, width, _ = frame.shape
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(frame_rgb)
            
            if results.detections is not None:
                for detection in results.detections:
                    xmin = int(detection.location_data.relative_bounding_box.xmin * width)
                    ymin = int(detection.location_data.relative_bounding_box.ymin * height)
                    w = int(detection.location_data.relative_bounding_box.width * width)
                    h = int(detection.location_data.relative_bounding_box.height * height)
                    if xmin < 0 and ymin < 0:
                        continue
                    
                    # cv2.rectangle(frame, (xmin, ymin), (xmin + w, ymin + h), (0, 255, 0), 5)
                    
                    face_image = frame[ymin : ymin + h, xmin: xmin + w]
                    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                    face_image = cv2.resize(face_image, (72, 72), interpolation=cv2.INTER_CUBIC)
                    
                    result = face_mask.predict(face_image)
                    # cv2.putText(frame, "{}".format(result), (xmin, ymin - 5), 1, 1.3, (218, 124, 176), 1, cv2.LINE_AA)
                    
                    if result[1] < 150:
                        color = (0, 255, 0) if LABELS[result[0]] == "Con_mascarilla" else (0, 0, 255)
                        cv2.putText(frame, "{}".format(LABELS[result[0]]), (xmin, ymin - 25), 2, 1, color, 1, cv2.LINE_AA)
                        cv2.rectangle(frame, (xmin, ymin), (xmin + w, ymin + h), color, 2)
            
            cv2.imshow("Frame", frame)
            k = cv2.waitKey(1)
            if k == 27:
                break
            
    cap.release()
    cv2.destroyAllWindows()


### Obtener Imagenes Negativas

In [19]:
nfolder = './n'
# nfolder = './faces/Sin_mascarilla'
nvideo = './old_negative.mp4'

if not os.path.exists(nfolder):
    os.makedirs(nfolder)
    print(f"Folder '{nfolder}' creado")

save_frames_from_video(nvideo, nfolder, frame_size=(64, 64), frame_interval=20)

Folder './n' creado
87 fotos guardadas en ./n


### Obtener Imagenes Positivas

In [20]:
pfolder = './p'
# nfolder = './faces/Con_mascarilla'
pvideo = './old_positive.mp4'

if not os.path.exists(pfolder):
    os.makedirs(pfolder)
    print(f"Folder '{pfolder}' creado")

save_frames_from_video(pvideo, pfolder, frame_size=(64, 64), frame_interval=25)

Folder './p' creado
70 fotos guardadas en ./p


### Generar Haarcasacde

In [ ]:
# dataset = './faces'
# dirlist = os.listdir(dataset)
# print("Archivos: ", dirlist)

# labels = []
# facesData = []
# label = 0

# for namedir in dirlist:
#     dirpath = dataset + "/" + namedir
    
#     for  filename in os.listdir(dirpath):
#         imagepath = dirpath + "/" + filename
#         print(imagepath)
#         image = cv2.imread(imagepath, 0)
        
#         facesData.append(image)
#         labels.append(label)
#     label += 1
    
# # print("Tag 0: ", np.count_nonzero(np.array(label) == 0))
# # print("Tag 1: ", np.count_nonzero(np.array(label) == 1))

# face_mask_train = cv2.face.LBPHFaceRecognizer_create()

# print("Trainning...")
# face_mask_train.train(facesData, np.array(labels))

# face_mask_train.write("cubrebocas.xml")
# print("modelo almacenado")

In [26]:
from PIL import Image
import os

def resize_images_in_folder(source_folder, target_size, output_folder):
    """
    Resizes all images in the specified folder to the target size and saves them to the output folder.

    :param source_folder: Path to the folder containing the images to be resized.
    :param target_size: A tuple (width, height) representing the new size of the images.
    :param output_folder: Path to the folder where resized images will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(source_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            img_path = os.path.join(source_folder, filename)
            img = Image.open(img_path)
            img = img.resize(target_size)
            img.save(os.path.join(output_folder, filename))


In [4]:
def count_files_in_folder(folder_path):
    try:
        files = os.listdir(folder_path)

        # Count the number of files
        file_count = len(files)

        return file_count

    except FileNotFoundError:
        print(f"Error: Folder '{folder_path}' not found.")
        return 0

folder_path = './p'
files_count = count_files_in_folder(folder_path)

print(f"There are {files_count} files in the folder '{folder_path}'.")


There are 260 files in the folder './p'.


In [28]:
resize_images_in_folder('../haarcascade/n72', (40, 40), '../haarcascade/n')
resize_images_in_folder('../haarcascade/p72', (40, 40), '../haarcascade/p')

### Utilizar cascade.xml Generado

In [29]:
test_video = 'test.mp4'
# cascade_path = './classifier/cascade.xml'
# cascade_path = './haarcascade_frontalface_alt.xml'
cascade_path = '../CascadeModels/cascade.xml'

if os.path.exists(cascade_path):
    buscando_a_nemo(test_video, cascade_path, scale_factor=10, min_neighbors=1000, min_size=200)
    # buscando_a_dori(test_video, cascade_path)